In [32]:
# Импорт библиотек
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import RFE, SelectKBest, f_regression

In [33]:
# Загрузка данных
data = pd.read_excel('data/data_ford_price.xlsx')

In [34]:
# Подготовка данных
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace=True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [35]:
# Метод RFE
estimator = LinearRegression()
rfe_selector = RFE(estimator, n_features_to_select=3, step=1)
rfe_selector.fit(X_train, y_train)
rfe_features = rfe_selector.get_feature_names_out()
print("Признаки, выбранные RFE:", rfe_features)

Признаки, выбранные RFE: ['year' 'cylinders' 'lat']


In [36]:
# Метод SelectKBest
kbest_selector = SelectKBest(f_regression, k=3)
kbest_selector.fit(X_train, y_train)
kbest_features = kbest_selector.get_feature_names_out()
print("Признаки, выбранные SelectKBest:", kbest_features)

Признаки, выбранные SelectKBest: ['year' 'cylinders' 'odometer']


In [37]:
# Обучение на RFE-признаках
X_train_rfe = X_train[rfe_features]
X_test_rfe = X_test[rfe_features]

model_rfe = LinearRegression()
model_rfe.fit(X_train_rfe, y_train)
y_pred_rfe = model_rfe.predict(X_test_rfe)
mae_rfe = mean_absolute_error(y_test, y_pred_rfe)
print(f"MAE для модели с RFE-признаками: {mae_rfe:.3f}")

MAE для модели с RFE-признаками: 5206.677


In [38]:
# Обучение на SelectKBest-признаках
X_train_kbest = X_train[kbest_features]
X_test_kbest = X_test[kbest_features]

model_kbest = LinearRegression()
model_kbest.fit(X_train_kbest, y_train)
y_pred_kbest = model_kbest.predict(X_test_kbest)
mae_kbest = mean_absolute_error(y_test, y_pred_kbest)
print(f"MAE для модели с SelectKBest-признаками: {mae_kbest:.3f}")

MAE для модели с SelectKBest-признаками: 4800.916


In [39]:
# Сравнение результатов
if mae_rfe < mae_kbest:
    print("\nЛучший метод: RFE")
    print(f"RFE показал MAE на {mae_kbest - mae_rfe:.3f} меньше, чем SelectKBest")
elif mae_kbest < mae_rfe:
    print("\nЛучший метод: SelectKBest")
    print(f"SelectKBest показал MAE на {mae_rfe - mae_kbest:.3f} меньше, чем RFE")
else:
    print("\nОба метода показали одинаковый результат")


Лучший метод: SelectKBest
SelectKBest показал MAE на 405.761 меньше, чем RFE


Лучший метод: SelectKBest\
SelectKBest показал MAE на 405.761 меньше, чем RFE